_____
***

<img width='700' src="https://user-images.githubusercontent.com/8030363/108961534-b9a66980-7634-11eb-96e2-cc46589dcb8c.png" style="vertical-align:middle">

***
***

**Author:** [TJCallahan](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=callahantiff@gmail.com)  
**GitHub Repository:** [PheKnowLator](https://github.com/callahantiff/PheKnowLator/wiki)  
**Current Release:** **[`v2.0.0`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**

<br>

**Objective:** Knowledge graphs provide meaningful ways to integrate heterogeneous biological data and represent complex biological mechanisms. This work seeks to explore the utility of incorporating existing knowledge of molecular mechanism from ontologies, publicly available data, and the literature to construct a biomedical knowledge graph that models unbiased molecular mechanisms of human disease.

<a target="_blank" href="https://user-images.githubusercontent.com/8030363/103158881-11813b00-4780-11eb-8b45-5063765e7645.png"> <img src="https://user-images.githubusercontent.com/8030363/103158881-11813b00-4780-11eb-8b45-5063765e7645.png"></a> 

(*Click Figure to Enlarge Image in Current Browser Tab*)

<br>

***
***

## Notebook Purpose
**Wiki Page:** **[`Release v2.0.0`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**

<br>

**Purpose:** This notebook serves as a `main` file for the PheKnowLator project. This scripts walks through this program step-by-step and generates the knowledge graph shown above. There is also a command line version of this file ([`main.py`](https://github.com/callahantiff/PheKnowLator/blob/master/main.py)). Please see the [README](https://github.com/callahantiff/PheKnowLator/blob/master/README.md) for more information.

<br>

**Assumptions:**
1. Hyperlinks to all downloaded and generated data sources are provided through [this](https://console.cloud.google.com/storage/browser/pheknowlator/release_v2.0.0?project=pheknowlator) dedicated Google Cloud Storage Bucket. <u>This includes examples, from prior builds, of the required input documents mentioned below</u>.     
2. Make sure that the following input documents have been constructed (see the [Dependencies Wiki](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies) for more information):  
  - [`resource_info.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/resource_info.txt)
  - [`ontology_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/ontology_source_list.txt)
  - [`edge_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/edge_source_list.txt)   

3. Prepare [relations](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data) and [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) files prior to running the scripts.  

4. Select a knowledge graph build type (i.e. `full`, `partial`, or `post-closure`) and construction method (i.e. `instance-based` or `subclass-based`).  

<br>

***
### Table of Contents
***
The three primary steps involved in building a knowledge graph are `Downloading Data Sources`, `Creating Edge Lists`, and `Building the knowledge graphs`.

* [Data Sources](#data-source)  
* [Create Edge Lists](#create-edges)  
* [Build Knowledge Graph](#build-kg)  

***

***

_____
### Set-Up Environment

In [1]:
# import needed libraries
import glob
import json
import pandas
import ray
import time

# import module
from pkt_kg import downloads, edge_list, knowledge_graph

***
## Download Data Sources <a class="anchor" id="data-source"></a>

**Wiki Page:** **[`Dependencies`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies)**  

**Purpose:**
This portion of this portion of the algorithm is to download:
1. [Download Ontology Data](#download-ontology-data)  
2. [Download Edge Data](#download-edge-data)   

<br>

**Input Files:**
  - [`resource_info.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/resource_info.txt)
  - [`ontology_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/ontology_source_list.txt)
  - [`edge_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/edge_source_list.txt)

<br>

**Assumptions:**  
- All sources used to construct our knowledge graph need to be preprocessed and ready to download prior to running this code. All mapping, filtering, and label data have been generated prior to this step. For assistance with creating these datasets, see the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb) Jupyter Notebook.  
- All downloaded and generated data sources for all PheKnowLator builds can be accessed through [this](https://console.cloud.google.com/storage/browser/pheknowlator/release_v2.0.0?project=pheknowlator) dedicated Google Cloud Storage Bucket.


***
***
### Ontology Data  <a class="anchor" id="download-ontology-data"></a>
Ontologies are the core data structure used when building PheKnowLator. In the figure above, ontology data are shown in yellow boxes.

In [2]:
from pkt_kg.downloads import OntData, LinkedData
from pkt_kg.edge_list import CreatesEdgeList
from pkt_kg.knowledge_graph import FullBuild, PartialBuild, PostClosureBuild

In [3]:
ont = OntData('resources/ontology_source_list.txt', 'resources/resource_info.txt')
#ont.writes_source_metadata_locally()

In [4]:
ont.parses_resource_file()

In [5]:
ont.source_list

{'disease': 'resources/ontologies/mondo_with_imports.owl',
 'napdichem': 'resources/ontologies/chebi_lite_merged_with_imports.owl',
 'protein': 'resources/ontologies/pr_with_imports.owl',
 'pathway': 'resources/ontologies/pw_with_imports.owl',
 'relation': 'resources/ontologies/ro_with_imports.owl',
 'go': 'resources/ontologies/go_with_imports.owl',
 'chemical': 'resources/ontologies/chebi_with_imports.owl',
 'phenotype': 'resources/ontologies/hp_with_imports.owl',
 'anatomy': 'resources/ontologies/uberon_with_imports.owl',
 'cell': 'resources/ontologies/clo_with_imports.owl',
 'genomic': 'resources/ontologies/so_with_imports.owl',
 'oae': 'resources/ontologies/oae_merged_with_imports.owl',
 'plant': 'resources/ontologies/po_with_imports.owl',
 'dideo': 'resources/ontologies/dideo_with_imports.owl'}

In [6]:
ont.data_files = ont.source_list
ont.generates_source_metadata()


*** Generating Metadata ***



100%|██████████| 14/14 [00:00<00:00, 37141.21it/s]


In [7]:
ont._writes_source_metadata_locally()

100%|██████████| 14/14 [00:00<00:00, 46938.65it/s]


In [8]:
ont.resource_info

['chemical-diseasetherapeutic|:;MESH_;|class-class|RO_0002606|http://purl.obolibrary.org/obo/|http://purl.obolibrary.org/obo/|t|1;4|0:./resources/processed_data/MESH_CHEBI_MAP.txt;1:./resources/processed_data/DISEASE_MONDO_MAP.txt|5;==;therapeutic|None',
 'chemical-disease|:;MESH_;|class-class|RO_0003302|http://purl.obolibrary.org/obo/|http://purl.obolibrary.org/obo/|t|1;4|0:./resources/processed_data/MESH_CHEBI_MAP.txt;1:./resources/processed_data/DISEASE_MONDO_MAP.txt|5;==;marker/mechanism|None',
 "chemical-gene|;MESH_;|class-entity|RO_0002434|http://purl.obolibrary.org/obo/|http://www.ncbi.nlm.nih.gov/gene/|t|1;4|0:./resources/processed_data/MESH_CHEBI_MAP.txt|9;affects;not in x|6;==;Homo sapiens::5;.startswith('gene');",
 'chemical-gobp|:;MESH_;GO_|class-class|RO_0002436|http://purl.obolibrary.org/obo/|http://purl.obolibrary.org/obo/|t|1;5|0:./resources/processed_data/MESH_CHEBI_MAP.txt|8;<=;1.04e-47|3;==;Biological Process',
 'chemical-gocc|:;MESH_;GO_|class-class|RO_0002436|http:

<br>

### Edge Data   <a class="anchor" id="download-edge-data"></a>
In PheKnowLator, classes are nodes that originate from ontologies. Class data sources are Linked Data sources that are used to create edges in the knowledge graph and thus can connect to other class data sources. Sometimes we want to add data that is not already part of an ontology. In that case, data either be added as an `instance` of an existing ontology class or as its own `owl:class` by being added to the knowledge graph as a `subclass` of an existing `owl:class`.

In [ ]:
#do this the first time to download all the edge data

## Data versions for original_data/: Jan 2022
## Data versions for processed_data/: Aug 2023
edges = LinkedData('resources/edge_source_list_TC.txt', 'resources/resource_info.txt')
edges.downloads_data_from_url()
#edges.writes_source_metadata_locally()

In [9]:
edges = LinkedData('resources/edge_source_list.txt', 'resources/resource_info.txt')

In [10]:
edges.parses_resource_file()

In [11]:
edges.source_list

{'chemical-diseasetherapeutic': 'resources/edge_data/chemical-disease_CTD_chemicals_diseases.tsv',
 'chemical-disease': 'resources/edge_data/chemical-disease_CTD_chemicals_diseases.tsv',
 'chemical-gene': 'resources/edge_data/chemical-gene_CTD_chem_gene_ixns.tsv',
 'chemical-gobp': 'resources/edge_data/chemical-gobp_CTD_chem_go_enriched.tsv',
 'chemical-gocc': 'resources/edge_data/chemical-gocc_CTD_chem_go_enriched.tsv',
 'chemical-gomf': 'resources/edge_data/chemical-gomf_CTD_chem_go_enriched.tsv',
 'chemical-pathway': 'resources/edge_data/chemical-pathway_ChEBI2Reactome_All_Levels.txt',
 'chemical-protein': 'resources/edge_data/chemical-protein_CTD_chem_gene_ixns.tsv',
 'chemical-phenotypetherapeutic': 'resources/edge_data/chemical-disease_CTD_chemicals_diseases.tsv',
 'chemical-phenotype': 'resources/edge_data/chemical-disease_CTD_chemicals_diseases.tsv',
 'disease-phenotype': 'resources/edge_data/disease-phenotype_phenotype.hpoa',
 'gene-disease': 'resources/edge_data/gene-disease_

In [12]:
edges.data_files = edges.source_list
edges.generates_source_metadata()


*** Generating Metadata ***



100%|██████████| 40/40 [00:00<00:00, 24877.25it/s]


In [13]:
edges._writes_source_metadata_locally()

100%|██████████| 40/40 [00:00<00:00, 60219.73it/s]


In [14]:
edges.source_list.keys()

dict_keys(['chemical-diseasetherapeutic', 'chemical-disease', 'chemical-gene', 'chemical-gobp', 'chemical-gocc', 'chemical-gomf', 'chemical-pathway', 'chemical-protein', 'chemical-phenotypetherapeutic', 'chemical-phenotype', 'disease-phenotype', 'gene-disease', 'gene-gene', 'gene-pathway', 'gene-phenotype', 'gene-protein', 'gene-rna', 'gobp-pathway', 'pathway-gocc', 'pathway-gomf', 'protein-anatomy', 'protein-catalyst', 'protein-cofactor', 'protein-cell', 'protein-gobp', 'protein-gocc', 'protein-gomf', 'protein-pathway', 'protein-protein', 'rna-anatomy', 'rna-cell', 'rna-protein', 'variant-disease', 'variant-gene', 'variant-phenotype', 'transporter-chemical', 'chemical-molecule', 'chemical-substrate', 'chemical-inhibitor', 'chemical-indication'])

<br>

***

## Create Edge Lists <a class="anchor" id="create-edges"></a>

**Wiki Page:** **[`Data Sources`](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources)**

<br>

**Purpose:** The code below will take the dictionaries of processed data described above and use them to create edge lists for each of the edge types specified in the [`resource_info.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/resource_info.txt). Each edge list will be appended to a nested dictionary (see details below).

<br>

**Assumptions:**  
1. All `ontology` and `edge` data sources have been downloaded.   

2. All code in the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb) Jupyter Notebook has been run. This Notebook contains code needed to generate all mapping, filtering, and label data.

<br>

**Output:** `Master_Edge_List_Dict.json`. Below is an example of what the `Master Edge Dictionary` contains for each processed resource:  
```python
master_edges = {'chemical-disease'  :
                {'source_labels'    : ';MESH_;',
                 'data_type'        : 'class-class',
                 'edge_relation'    : 'RO_0002606',
                 'uri'              : ('http://purl.obolibrary.org/obo/',
                                       'http://purl.obolibrary.org/obo/'),
                 'delimiter'        : '#',
                 'column_idx'       : '1;4',
                 'identifier_maps'  : '0:./MESH_CHEBI_MAP.txt;1:disease-dbxref-map',
                 'evidence_criteria': "5;!=;' ",
                 'filter_criteria'  : 'None',
                 'edge_list'        : ['...']}
```

***

In [15]:
# set-up environment for parallel processing -- even if running program serially these steps are needed
import psutil
cpus = psutil.cpu_count(logical=True)
ray.init()

/home/sanya/.conda/envs/sanya_kg/lib/python3.6/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


{'node_ip_address': '10.226.40.229',
 'raylet_ip_address': '10.226.40.229',
 'redis_address': '10.226.40.229:6379',
 'object_store_address': '/tmp/ray/session_2023-09-11_02-34-44_567749_1571370/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2023-09-11_02-34-44_567749_1571370/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2023-09-11_02-34-44_567749_1571370',
 'metrics_export_port': 59166,
 'node_id': '314e51cf2d20f05aba3b960ef008b365750df9807562327e0fe55e85'}

In [16]:
# combine data sources
combined_edges = dict(edges.data_files, **ont.data_files)
resource_info_loc = './resources/resource_info.txt'

# initialize edge dictionary class
master_edges = CreatesEdgeList(data_files=combined_edges, source_file=resource_info_loc)
master_edges.runs_creates_knowledge_graph_edges(source_file=resource_info_loc, data_files=combined_edges, cpus=cpus)

(pid=1571738) Finished Edge: gene-protein (gene = 19324, protein = 19129); 19527 unique edges
(pid=1571738) Finished Edge: rna-protein (rna = 28220, protein = 18936); 28220 unique edges
(pid=1571743) Finished Edge: gene-phenotype (gene = 6787, phenotype = 1595); 24695 unique edges
(pid=1571733) Finished Edge: gene-disease (gene = 5059, disease = 4484); 13344 unique edges
(pid=1571742) Finished Edge: gene-pathway (gene = 10369, pathway = 1860); 107009 unique edges
(pid=1571739) Finished Edge: chemical-pathway (chemical = 2247, pathway = 2243); 29988 unique edges
(pid=1571739) Finished Edge: protein-cofactor (protein = 1584, cofactor = 44); 1998 unique edges
(pid=1571739) Finished Edge: chemical-inhibitor (chemical = 129, inhibitor = 29); 273 unique edges
(pid=1571746) Finished Edge: disease-phenotype (disease = 12100, phenotype = 10129); 444807 unique edges
(pid=1571743) Finished Edge: rna-cell (rna = 10052, cell = 4); 33152 unique edges
(pid=1571742) Finished Edge: rna-anatomy (rna = 2

**Preview Master Edge Data**  
Generate a table that includes each `edge-type`, its primary `relation`, example identifiers, and count of unique edges.

In [17]:
# # read in master edge dictionary
master_edges = json.load(open('resources/Master_Edge_List_Dict.json', 'r'))
master_edges.keys()

dict_keys(['gobp-pathway', 'chemical-phenotypetherapeutic', 'gene-disease', 'protein-pathway', 'protein-gocc', 'rna-anatomy', 'transporter-chemical', 'rna-protein', 'variant-phenotype', 'pathway-gocc', 'chemical-gobp', 'chemical-gene', 'chemical-gomf', 'chemical-diseasetherapeutic', 'gene-gene', 'variant-gene', 'chemical-phenotype', 'protein-gobp', 'protein-catalyst', 'protein-anatomy', 'chemical-indication', 'disease-phenotype', 'protein-cofactor', 'chemical-substrate', 'chemical-inhibitor', 'gene-rna', 'chemical-disease', 'chemical-pathway', 'protein-protein', 'protein-cell', 'variant-disease', 'protein-gomf', 'chemical-protein', 'gene-phenotype', 'chemical-gocc', 'rna-cell', 'chemical-molecule', 'gene-protein', 'gene-pathway', 'pathway-gomf'])

In [18]:
# read in relation data
relation_data = open('./resources/relations_data/RELATIONS_LABELS.txt').readlines()
relation_dict = {x.split('\t')[0]: x.split('\t')[1].strip('\n') for x in relation_data}
len(relation_dict.keys())

685

In [19]:
##add substrate of relation
relation_dict['DIDEO_00000041'] = 'is substrate of'

In [20]:
# print basic stats on each resource
edge_data = [[key,
              relation_dict[master_edges[key]['edge_relation']],
              ', '.join(master_edges[key]['edge_list'][0]),
              len(master_edges[key]['edge_list'])]
             for key in master_edges.keys()]

# convert dict to pandas df for nice printing
df = pandas.DataFrame(edge_data, columns = ['Edge Type', 'Relation', 'Example Edge', 'Unique Edges']) 
df

,Edge Type,Relation,Example Edge,Unique Edges
0,gobp-pathway,realized in response to,"GO_0016567, R-HSA-8866654",672
1,chemical-phenotypetherapeutic,is substance that treats,"CHEBI_701, HP_0001250",37369
2,gene-disease,causes or contributes to condition,"2, MONDO_0010200",13344
3,protein-pathway,participates in,"PR_A0A075B6P5, R-HSA-109582",117707
4,protein-gocc,located in,"PR_A0A024RBG1, GO_0005829",82524
5,rna-anatomy,located in,"ENST00000359318, UBERON_0001987",474073
6,transporter-chemical,transports,"PR_Q9NPD5, CHEBI_38545",91
7,rna-protein,ribosomally translates to,"ENST00000499023, PR_O43660",28220
8,variant-phenotype,causes or contributes to condition,"rs527236137, HP_0000556",4855
9,pathway-gocc,has component,"R-HSA-166753, GO_0005576",16014


In [21]:
df.to_csv('resources/edge_data/edge_counts.tsv', sep='\t', index=False)

<br><br>

***

## Build Knowledge Graph  <a class="anchor" id="build-kg"></a>
**Wiki Pages:**  
- **[`KG-Construction`](https://github.com/callahantiff/PheKnowLator/wiki/KG-Construction)**  
- **[`relations-data`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data)**  
- **[`node-metadata`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata)** 

**Jupyter Notebooks:**  
- [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb)  
[`Ontology_Cleaning.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Ontology_Cleaning.ipynb)  


<br>

**Assumptions:**  
- <u>Construction Approach</u>. If using the `subclass-based` construction approach, please make sure that a `pickled` dictionary mapping each non-ontology data node to an existing ontology class is created and added to the `./resources/knowledge_graph` directory (please see [here](https://github.com/callahantiff/PheKnowLator/tree/master/resources/knowledge_graphs#construction-method) for additional information).   
- <u>Relations Data</u>. If inverse relation data is going to be used to build the knowledge graph, that it has been generated and added to the `./resources/relations_data` directory (please see [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/relations_data/README.md) for additional information).  
- <u>Node Metadata</u>. If node metadata is going to be used to build the knowledge graph, that it has been generated and added to the `./resources/node_metadata` directory (please see [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/node_data/README.md) for additional information).  
- <u>Decoding OWL Semantics</u>. If decoding OWL-Semantics, please make sure to provide a list of owl:Property types to keep is created and added to the `./resources/knowledge_graph` directory (please see [here](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0) for additional information). 

<br>

**Input:** 
- `Master_Edge_List_Dict.json`  
- Directory of relations data sources - see [here](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data) for more information
- Directory of node data sources - see [here](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) for more information

<br>

**Output:** Please see [`Release v2.0.0 Wiki`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0) for access to all generated output files.   
- `Knowledge Graph` (`.owl` and Networkx MultiDiGraph `.pkl`)  
- `Class Instance URI-UUID Map` (if "instance" construction approach)   
- `Triple List - Integer`  
- `Triple List - Identifier`  
- `Node Integer-Identifier Map`  
- `Node Attribute Data`  

<br>

The process to build the knowledge graph is somewhat time consuming and can be broken into the following steps:  

1. Merge Ontologies. See [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/ontologies/README.md) for additional information on how to preprocess the ontologies prior to merging them.    

2. Create Edges. Add edge lists to merged ontologies.  

3. Add Inverse Relations and Node Data. See the [Dependencies](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies) Wiki page for details on how to construct these resources.  

4. Filter OWL Semantics. Filter the knowledge graph with the goal of removing all edges that contain entities that are needed to support owl semantics, but are not biologically meaningful (please see [here](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0) for additional information).

5. Save Edge Lists and Node Metadata. Several versions of the knowledge graph are saved, including: the full knowledge graph (`owl` or Networkx MultiDiGraph `pickle`), triple lists (i.e. integer index and identifier labeled edge lists with a dictionary that maps between the integer indices and node identifiers), and a file of metadata (i.e. identifiers, labels, synonyms, and descriptions) for all nodes in the knowledge graph.  

<br>

**‼ IMPORTANT:**  
- The file containing the merged ontologies is quite large and can take up to 30 minutes to read in.  This is not a limitation of the code directly, but rather a function of the [`RDFLib Library`](https://github.com/RDFLib). While there are other ways to read in this data, we maintain reliance on this library as it is the most user-friendly for non-RDF users.   
- If you'd like to include [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) when building the knowledge graph, please hold off on building the knowledge graph until you have generated the node data. For details on how to do this see the [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) section of the `Dependencies` Wiki Page or help for help with generating the data, please see the []() section of the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb) Jupyter Notebook.

***


In [22]:
# specify input arguments
build = 'full'
construction_approach = 'instance'
add_node_data_to_kg = 'yes'
add_inverse_relations_to_kg = 'yes'
decode_owl_semantics = 'yes'
kg_directory_location = './resources/knowledge_graphs'


In [23]:
# construct knowledge graphs
# run sed command - sed -i 's/_:genid/_genid/g' resources/knowledge_graphs/PheKnowLator_MergedOntologies.owl
if build == 'partial':
    kg = PartialBuild(construction=construction_approach,
                      node_data=add_node_data_to_kg,
                      inverse_relations=add_inverse_relations_to_kg,
                      decode_owl=decode_owl_semantics,
                      cpus=cpus,
                      write_location=kg_directory_location)
elif build == 'post-closure':
    kg = PostClosureBuild(construction=construction_approach,
                          node_data=add_node_data_to_kg,
                          inverse_relations=add_inverse_relations_to_kg,
                          decode_owl=decode_owl_semantics,
                          cpus=cpus,
                          write_location=kg_directory_location)
else:
    kg = FullBuild(construction=construction_approach,
                   node_data=add_node_data_to_kg,
                   inverse_relations=add_inverse_relations_to_kg,
                   decode_owl=decode_owl_semantics,
                   cpus=cpus,
                   write_location=kg_directory_location)

kg.construct_knowledge_graph()
ray.shutdown()


### Starting Knowledge Graph Build: FULL ###
*** Loading Relations Data ***
Loading and Processing Relation Data
*** Loading Merged Ontologies ***
Merged Ontologies Graph Stats: 23996958 triples, 8901531 nodes, 330 predicates, 835991 classes, 36 individuals, 867 object props, 700 annotation props
*** Loading Node Metadata Data ***
Loading and Processing Node Metadata

Extracting Class and Relation Metadata


100%|██████████| 867/867 [00:00<00:00, 3536.79it/s]


*** Splitting Graph ***
Adding Namespace to BNodes
Creating Logic and Annotation Subsets of Graph


100%|██████████| 2731462/2731462 [36:45<00:00, 1238.73it/s] 


Annotation Assertions (n=17718241 Triples)
Creating Logic Graph (n=6278717 Triples)


...

Depduplicating File: ./resources/knowledge_graphs/PheKnowLator_v3.1.1_full_instance_inverseRelations_OWL_AnnotationsOnly.nt


100%|██████████| 18688005/18688005 [01:45<00:00, 177815.17it/s]


Depduplicating File: ./resources/knowledge_graphs/PheKnowLator_v3.1.1_full_instance_inverseRelations_OWL_LogicOnly.nt


100%|██████████| 27055279/27055279 [03:31<00:00, 127885.03it/s]


Merging Files: ./resources/knowledge_graphs/PheKnowLator_v3.1.1_full_instance_inverseRelations_OWL_AnnotationsOnly.nt and ./resources/knowledge_graphs/PheKnowLator_v3.1.1_full_instance_inverseRelations_OWL_LogicOnly.nt


Loading Full (Logic + Annotation) Graph

Deriving Stats

Processing pkt-namespaced BNodes in Full (Logic + Annotation) graph
Identifying BNodes with Namespace: https://github.com/callahantiff/PheKnowLator/pkt/bnode/
Identifying BNodes
Removing Namespace from BNodes
Finalizing Updated Graph
Identifying BNodes with Namespace: https://github.com/callahantiff/PheKnowLator/pkt/
Identifying BNodes
Removing Namespace from BNodes
Finalizing Updated Graph
Applying OWL API Formatting to Knowledge Graph OWL File

Full (Logic + Annotation) Graph Stats: 45743284 triples, 17351837 nodes, 356 predicates, 1270547 classes, 7038946 individuals, 867 object props, 700 annotation props


<br>

***
***

```
@misc{callahan_tj_2019_3401437,
  author       = {Callahan, TJ},
  title        = {PheKnowLator},
  month        = mar,
  year         = 2019,
  doi          = {10.5281/zenodo.3401437},
  url          = {https://doi.org/10.5281/zenodo.3401437}
}
```